# Monitoraggio dei bias di modelli Amazon SageMaker: equità e spiegabilità con SageMaker Clarify

Amazon SageMaker Clarify aiuta a migliorare i modelli di machine learning rilevando potenziali bias e aiutando a spiegare in che modo tali modelli effettuano previsioni. La funzionalità di equità e spiegabilità di SageMaker Clarify compie un ulteriore passo per consentire ai clienti AWS di creare modelli di machine learning affidabili e comprensibili. Il prodotto include gli strumenti utili a eseguire le seguenti attività.

- Misurare i bias che possono verificarsi in ciascun passaggio del ciclo di vita del machine learning (raccolta dei dati, addestramento e messa a punto del modello, monitoraggio dei modelli di machine learning distribuiti per inferenza).
- Generare report di governance del modello per informare i team di rischio e conformità e le autorità di regolamentazione.
- Fornire spiegazioni di dati, modelli e monitoraggio utilizzati per valutare le previsioni.

## Importazioni

In [ ]:
import os
import copy
import json
import random
import time
import datetime
import boto3
import pandas as pd

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor

from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

### Configurazione per questa attività

In [ ]:
# Sostituisci <bucket_name> di seguito con il valore modelArtifactBucket a sinistra della pagina delle istruzioni
bucket = '<bucket_name>'

# Ottieni il nome dell'endpoint di produzione
endpoint_name = boto3.Session().client('sagemaker').list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']

# Ottieni il nome del modello di produzione
model_name = boto3.Session().client('sagemaker').list_models(SortBy='CreationTime')['Models'][0]['ModelName']


print(f"Bucket Name :{bucket}")
print(f"Production Endpoint Name:{endpoint_name}")
print(f"Production Model Name:{model_name}")

In [ ]:
role = get_execution_role()
print(f"RoleArn: {role}")

sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

region = sagemaker_session.boto_region_name
print(f"AWS region: {region}")

In [ ]:
print(f"Demo Bucket: {bucket}")
prefix = f'sagemaker/DEMO-ClarifyModelMonitor-{datetime.now():%Y%m%d%H%M%S}'
s3_key = f"s3://{bucket}/{prefix}"
print(f"S3 key: {s3_key}")

code_prefix = '{}/code'.format(prefix)
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f'{s3_key}/datacapture'
s3_report_path = f'{s3_key}/reports'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

endpoint_instance_count = 1
endpoint_instance_type = "ml.m5.xlarge"

### File dei dati

Leggi i file dei dati

In [ ]:
test_dataset = "data/test.csv"
validation_dataset = "data/train.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

### Abilita l'acquisizione di dati sull'endpoint

Per abilitare l'acquisizione dei dati per il monitoraggio dei processi, specifica qui la nuova opzione di acquisizione denominata DataCaptureConfig, che abilita l'acquisizione del payload richiesto e del payload di risposta all'endpoint. La configurazione di acquisizione si applica a tutte le varianti. 

L'operazione richiede in genere 10-15 minuti.

**Nota**: può essere automatizzata mediante CloudFormation (reviewer - please suggest)

In [ ]:
data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path,
                        kms_key_id=None,
                        capture_options=["REQUEST", "RESPONSE"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"])



# Adesso occorre applicare la nuova configurazione e attenderne il completamento
predictor = Predictor(endpoint_name=endpoint_name)
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sagemaker_session.wait_for_endpoint(endpoint=endpoint_name)

### Test dell'endpoint con traffico di esempio

In [ ]:
# Qui scegliamo casualmente sottoinsiemi di dati dal set di dati di test.
import itertools

shape = pd.read_csv(test_dataset)

a = [10*i for i in range(3)]
b = [10+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data = shape.drop(shape.columns[[0]],axis=1)
test_data = test_data.iloc[indices]
test_data_with_label = shape.iloc[indices]

In [ ]:
test_data.to_csv("data/data-test.csv",index=False,header=False)
test_data_with_label.to_csv("data/data-test-label.csv",index=False,header=False)

In [ ]:
# Richiama l'endpoint con i dati per un test rapido
print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")
predictions = ''

with open('data/data-test.csv', 'r') as f:
    for row in f:
        print(".", end="", flush=True)
        payload = row.rstrip('\n')
        response = sagemaker_runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=payload)
        predictions = ','.join([predictions, response['Body'].read().decode('utf-8')])
        time.sleep(0.5)

predictions = predictions.replace('\n','')
predictions = predictions.split(",")
predictions.pop(0)
print("="*20)
print(predictions)
print("Done!") 

## Visualizza i dati acquisiti

Ora elenca i file dei dati di acquisizione archiviati in Amazon S3. Dovresti vedere file diversi da periodi di tempo diversi organizzati in base all'orario in cui si è verificata l'invocazione. Ecco il formato del percorso di Amazon S3:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
print(result)
str_error = "waiting for the file to load..."
while str_error:
    try:
        capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
        print(capture_files)
        str_error=None
    except:
        print(str_error)
        time.sleep(10)
        contine
print("Found Capture Files:")
print("\n ".join(capture_files))

Successivamente, visualizza il contenuto di un singolo file di acquisizione. Qui dovresti vedere tutti i dati acquisiti in un file specifico in formato JSON di Amazon SageMaker. Dai uno sguardo veloce alle prime righe del file acquisito.

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

Infine, ecco di seguito il contenuto di una singola riga in un file in formato JSON affinché tu possa osservarlo meglio.

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

### Configurazione di Amazon Sagemaker Clarify

**Nota**: gli esempi di codice seguenti sono estratti dal [codice di esempio](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker_processing/fairness_and_explainability/fairness_and_explainability.ipynb) in Amazon SageMaker. Alcuni codici sono aggiornati per i requisiti del laboratorio.

In [ ]:
# Esegui il processore SageMaker Clarify utilizzando il comando seguente
from sagemaker import clarify
clarify_processor = clarify.SageMakerClarifyProcessor(role=role,
                                                      instance_count=1,
                                                      instance_type='ml.m5.xlarge',
                                                      sagemaker_session=sagemaker_session)

Carica il test e i set di dati di addestramento in S3. Nei prossimi passaggi saranno utilizzati _train_uri_ e _test_uri_ per estrarre i dati direttamente da dataconfig e modelconfig di S3.

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = S3Uploader.upload('data/train.csv', 's3://{}/{}'.format(bucket, prefix))
train_input = TrainingInput(train_uri, content_type='csv')
test_uri = S3Uploader.upload('data/test.csv', 's3://{}/{}'.format(bucket, prefix))

### Rilevamento di bias

SageMaker Clarify ti aiuta a rilevare eventuali bias precedenti o successivi all'addestramento utilizzando una serie di metriche.

### DataConfig e ModelConfig in scrittura

Un oggetto **DataConfig** comunica a SageMaker Clarify alcune informazioni di base sull'I/O dei dati. Specificherai dove trovare il set di dati di input, dove archiviare l'output, la colonna di destinazione (etichetta), i nomi delle intestazioni e il tipo di set di dati.

In [ ]:
bias_report_output_path = 's3://{}/{}/clarify-bias'.format(bucket, prefix)
bias_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                      s3_output_path=bias_report_output_path,
                                      label='Churn',
                                      headers=all_headers,
                                      dataset_type='text/csv')

Un oggetto **ModelConfig** comunica informazioni sul modello addestrato. Per evitare traffico aggiuntivo ai modelli di produzione, SageMaker Clarify configura e interrompe un endpoint dedicato durante l'elaborazione.

- **instance_type** e **instance_count** specificano il tipo di istanza preferita e il conteggio istanze utilizzato per eseguire il modello durante l'elaborazione di SageMaker Clarify. Il set di dati di test è piccolo, pertanto è sufficiente una singola istanza standard per eseguire l'esempio. Se disponi di un set di dati più ampio, potresti velocizzare utilizzando un tipo di istanza migliore oppure aggiungere più istanze così da abilitare la parallelizzazione di Spark.

- **accept_type** indica il formato del payload di risposta dell'endpoint e **content_type** il formato del payload di richiesta all'endpoint.

In [ ]:
model_config = clarify.ModelConfig(model_name=model_name,
                                   instance_type='ml.m5.xlarge',
                                   instance_count=1,
                                   accept_type='text/csv',
                                   content_type='text/csv')

**ModelPredictedLabelConfig** fornisce informazioni sul formato delle previsioni. Il modello foresta casuale restituisce le probabilità dei campioni, pertanto SageMaker Clarify richiama l'endpoint e utilizza probability_threshold per convertire la probabilità in etichette binarie per l'analisi dei bias. Si intende superiore alla soglia una previsione con valore di etichetta pari a 1 e inferiore o uguale con valore di etichetta pari a 0.

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

### BiasConfig in scrittura
SageMaker Clarify deve inoltre conoscere quali sono le colonne sensibili (facet), quali potrebbero essere le caratteristiche sensibili (facet_values_or_threshold) e quali sono i risultati auspicabili (label_values_or_threshold). SageMaker Clarify può gestire i dati sia categoriali sia continui per facet_values_or_threshold e per label_values_or_threshold. In questo caso, stiamo usando i dati categoriali.

Specifichiamo questa informazione nell'API BiasConfig. Il risultato positivo qui è Churn (Abbandono) = 0, _Account Length_ è una categoria sensibile e gli intervistati _Day Calls_ sono un gruppo sensibile. group_name viene utilizzato per formare sottogruppi per la misurazione.

In [ ]:
bias_config = clarify.BiasConfig(label_values_or_threshold=[0],
                                facet_name='Account Length',
                                facet_values_or_threshold=[100],
                                group_name='Day Calls')

**Bias precedenti all'addestramento**

I bias possono essere nei dati prima che abbia luogo qualsiasi addestramento di modelli. L'ispezione dei dati in cerca di bias prima dell'addestramento può aiutare a rilevare eventuali lacune nella raccolta dei dati, a informare l'ingegneria delle funzionalità e a comprendere i bias sociali che i dati possono riflettere.

Il calcolo delle metriche relative ai bias precedenti all'addestramento non richiede un modello addestrato.

**Bias successivi all'addestramento**

Il calcolo delle metriche relative ai bias successivi all'addestramento richiede a sua volta un modello addestrato.

I dati di addestramento non distorti (come determinato dai concetti di equità misurati dalla metrica di bias) possono comunque comportare previsioni distorte del modello dopo l'addestramento. Che ciò avvenga dipende da diversi fattori tra cui le scelte di iperparametri.

Puoi eseguire le opzioni separatamente con run_pre_training_bias() e run_post_training_bias() oppure contemporaneamente con run_bias() come mostrato di seguito.

In [ ]:
clarify_processor.run_bias(data_config=bias_data_config,
                           bias_config=bias_config,
                           model_config=model_config,
                           model_predicted_label_config=predictions_config,
                           pre_training_methods='all',
                           post_training_methods='all')

**Visualizzazione del report sui bias**

Puoi visualizzare i risultati in Studio, nella scheda relativa agli esperimenti.

<img src="./recordings/bias_report.gif">

Ciascuna metrica di bias presenta spiegazioni dettagliate con esempi da provare.

<img src="./recordings/bias_detail.gif">

Puoi inoltre riepilogare i risultati in un'utile tabella.

<img src="./recordings/bias_report_chart.gif">



Se non sei ancora un utente di Studio, puoi accedere al report sui bias nei formati PDF, HTML e IPYNB nel seguente bucket S3:

In [ ]:
bias_report_output_path

### Spiegazione delle previsioni

Le esigenze di attività in espansione e i regolamenti legislativi possono richiedere una spiegazione sul motivo per cui un modello ha preso una determinata decisione. SageMaker Clarify utilizza SHAP per spiegare il contributo di ciascuna caratteristica di input nella decisione finale.

L'algoritmo kernel SHAP richiede un riferimento, noto anche come set di dati sul contesto. Il tipo di set di dati di riferimento deve essere quello già indicato in  dataset_type di DataConfig e i campioni di riferimento dovranno includere solamente le caratteristiche. Per definizione, la base di riferimento deve essere un URI S3 che punta al file con il set di dati oppure un elenco di esempi in linea. Scegliamo il secondo in questo caso e collochiamo il primo campione del set di dati di test nell'elenco.

In [ ]:
shap_config = clarify.SHAPConfig(baseline=[test_data.iloc[0].values.tolist()],
                                 num_samples=15,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=train_uri,
                                s3_output_path=explainability_output_path,
                                label='Churn',
                                headers=all_headers,
                                dataset_type='text/csv')

In [ ]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)

#### Visualizzazione del report di spiegabilità
Come per il report sui bias, puoi visualizzare i risultati in Studio, nella scheda relativa agli esperimenti.


<img src="./recordings/explainability_detail.gif">

La scheda Model Insights (Informazioni sul modello) contiene collegamenti diretti al report e alle informazioni sul modello.

Se non sei ancora un utente di Studio, come per il report sui bias, puoi accedere a questo report nel seguente bucket S3:

In [ ]:
explainability_output_path

### Eliminazione

In ultimo, non dimenticare di eliminare le risorse configurate e utilizzate per questa dimostrazione.

In [ ]:
session.delete_model(model_name)